## Wprowadzenie do **dbzero** (3/12): Badanie ograniczeń

In [ ]:
import dbzero as db0
from mem_charts import mem_usage_chart, random_string
from bokeh.io import show, output_notebook
import concurrent.futures

Zaimportowaliśmy pakiet 'bokeh' do wizualizacji obecnego wykorzystania pamięci na wykresie.

Stwórzmy również executor, aby móc uruchamiać zadania Pythona w tle (w oddzielnym wątku).

In [ ]:
executor = concurrent.futures.ThreadPoolExecutor(max_workers=1)

Wykres poniżej przedstawia żywe wykorzystanie pamięci przez obecny proces, odświeżane co 1 sekundę.

In [ ]:
output_notebook(resources=None, verbose=False, hide_banner=True)
show(mem_usage_chart, notebook_url="http://127.0.0.1:8888", port=8889)

Zobaczmy najpierw, jak rośnie wykorzystanie pamięci podczas uruchamiania zwykłego kodu Pythona. Dodajemy 50 000 losowych elementów tekstowych do zwykłej listy Pythona w 100 iteracjach. To łącznie 5 milionów dodanych elementów danych. Zobaczmy, jak to działa...

In [ ]:
result = []
def generate_sequence(result, length, batch):
    for _ in range(length):
        result.extend([random_string() for _ in range(batch)])
    print("Task finished")

In [ ]:
task = executor.submit(generate_sequence, result, length=100, batch = 50000)

Pamięć po prostu nadal rośnie i rośnie i ostatecznie zakończyłaby proces błędem "brak pamięci". W Pythonie możemy zwolnić pamięć poprzez zwykłe opróżnienie listy.

In [ ]:
result = []

### Dobrze, więc jak **dbzero** różni się w tej kwestii?
W **dbzero** pracujesz przez większość czasu jak ze zwykłym kodem Pythona, ale nie musisz już martwić się o ograniczenia pamięci. To prawda, nawet jeśli masz do czynienia z terabajtami danych, Twój proces nigdy nie przekroczy limitów, które sam zdefiniujesz.

Po inicjalizacji **dbzero** zajmie niewielką ilość Twojej pamięci...

In [ ]:
db0.init(dbzero_root = "/dbzero", prefix = "data")

Ale możesz kontrolować, ile dodatkowej pamięci używa, wywołując metodę 'set_cache_size'.

In [ ]:
db0.set_cache_size(128 << 20)

Powtórzmy teraz test używając db0.list (obiekt listy w przestrzeni dbzero). Obserwuj uważnie, jak wykorzystanie pamięci zatrzymuje się w pewnym momencie (gdy zostanie osiągnięty zdefiniowany limit cache) i nie rośnie bez względu na to, ile danych umieścisz w swojej liście.

In [ ]:
db0_result = db0.list()

In [ ]:
def db0_generate_sequence(result, length, batch):
    for _ in range(length):
        result.extend([random_string() for _ in range(batch)]) 
    print("Task finished")

In [ ]:
task = executor.submit(db0_generate_sequence, db0_result, length=100, batch = 50000)

In [ ]:
print(len(db0_result))
db0_result[12313]

### No więc gdzie w rzeczywistości są przechowywane dane w tym przypadku?
dbzero implementuje algorytmy wymiany pamięci. Pobiera dane z chmury lub, w przypadku wersji lokalnej, z systemu plików na zasadzie "według potrzeby" i przechowuje je w lokalnym cache, aby umożliwić szybki dostęp w przyszłości. Proces jest całkowicie przezroczysty dla programisty.

In [ ]:
db0.close()